In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# import the MNIST dataset
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms

from sklearn.model_selection import train_test_split

# from torchdiffeq import odeint, odeint_adjoint

import numpy as np

import matplotlib.pyplot as plt

from tqdm.notebook import trange

# use the ball dataset
from src.utils.utils import add_spatial_encoding, stack_dataset
from src.data.generate import create_gaussian_dataset
from src.models.vae import VAE, VAEwithResNet, custom_loss

In [2]:
# dataset = create_gaussian_dataset(r_min=3.0, r_max=3.0, n_samples=1000, size=28, margin=2, n_balls=1)
# dataset = add_spatial_encoding(dataset)
# # dataset = stack_dataset(dataset)

# train_dataset, test_dataset = train_test_split(dataset, test_size=0.3)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [3]:
# Download the MNIST dataset if it is not present
mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

train_dataset = stack_dataset(mnist_train)
test_dataset = stack_dataset(mnist_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)


/opt/homebrew/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [14]:
in_channels = 3
vae = VAEwithResNet(n_latent=7, dec_in_channels=3)
loss_fn = custom_loss()

In [15]:
test_input = torch.zeros(10, in_channels, 28, 28)
print(len(vae(test_input)), vae(test_input)[0].shape, vae(test_input)[1].shape, vae(test_input)[2].shape)  


3 torch.Size([10, 3, 28, 28]) torch.Size([10, 7]) torch.Size([10, 7])


/opt/homebrew/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [16]:
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
# optimmizer = torch.optim.RMSprop(vae.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9999)

In [ ]:
# print(vae.generate(1).shape)
vae.fit(train_loader, test_loader, optimizer, scheduler, loss_fn, epochs=100, display_step=10, n_plot=2)

In [71]:
# torch.save(vae.state_dict(), 'models/VAE/vae_resnet18_1_Ball_latent_4.pth')